In [1]:
!ls ../

COVID-19                  convert_to_json.py        scripts
Snakefile                 coordinates.csv           worldbank_population_data
cntry_stat.json           notebooks
combined_ts.csv           pop.csv


In [16]:
import pandas as pd
from venn import venn

ts = '../combined_ts.csv'
pop = '../pop.csv'

In [10]:
cntry_from_ts = set(pd.read_csv(ts).country)
cntry_from_pop = set(pd.read_csv(pop).country)

In [30]:
from pprint import pprint

print("Number of countries in the COVID-19 data:", len(cntry_from_ts))
print("Number of countries in the population data:", len(cntry_from_pop))

print("Intersection: ", len(cntry_from_ts & cntry_from_pop))
print
print("Only in the COVID-19 data:", sorted(cntry_from_ts - cntry_from_pop))
print
pprint("Only in the pop data:")
pprint(sorted(cntry_from_pop - cntry_from_ts))

Number of countries in the COVID-19 data: 90
Number of countries in the population data: 264
Intersection:  77
Only in the COVID-19 data: ['Egypt', 'Hong Kong', 'Iran', 'Macau', 'Mainland China', 'Others', 'Palestine', 'Russia', 'Saint Barthelemy', 'South Korea', 'Taiwan', 'UK', 'US']
'Only in the pop data:'
['Albania',
 'American Samoa',
 'Angola',
 'Antigua and Barbuda',
 'Arab World',
 'Aruba',
 'Bahamas, The',
 'Bangladesh',
 'Barbados',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Botswana',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cameroon',
 'Caribbean small states',
 'Cayman Islands',
 'Central African Republic',
 'Central Europe and the Baltics',
 'Chad',
 'Channel Islands',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Cuba',
 'Curacao',
 'Cyprus',
 'Djibouti',
 'Dominica',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East As

In [33]:
open('temp.txt', 'w').write('\n'.join(sorted(cntry_from_pop - cntry_from_ts)))

2801

In [52]:
df = pd.concat([pd.read_csv('../pop_raw.csv'), pd.read_csv('../pop_addition.csv')])
df.tail()

,country,country_code,population,region,income
260,"Yemen, Rep.",YEM,28498687.0,Middle East & North Africa,Low income
261,South Africa,ZAF,57779622.0,Sub-Saharan Africa,Upper middle income
262,Zambia,ZMB,17351822.0,Sub-Saharan Africa,Lower middle income
263,Zimbabwe,ZWE,14439018.0,Sub-Saharan Africa,Lower middle income
0,Taiwan,TWN,23780000.0,East Asia & Pacific,High income


In [40]:
conv = pd.read_csv('../pop_conversion.csv')
conv

,worldbank,covid19
0,"Egypt, Arab Rep.",Egypt
1,"Hong Kong SAR, China",Hong Kong
2,"Iran, Islamic Rep.",Iran
3,"Macao SAR, China",Macau
4,China,Mainland China
5,West Bank and Gaza,Palestine
6,Russian Federation,Russia
7,"Korea, Rep.",South Korea
8,United Kingdom,UK
9,United States,US


In [43]:
conv.set_index('worldbank')

,covid19
worldbank,
"Egypt, Arab Rep.",Egypt
"Hong Kong SAR, China",Hong Kong
"Iran, Islamic Rep.",Iran
"Macao SAR, China",Macau
China,Mainland China
West Bank and Gaza,Palestine
Russian Federation,Russia
"Korea, Rep.",South Korea
United Kingdom,UK


In [60]:
conv = conv.set_index('worldbank').to_dict()['covid19']

AttributeError: 'dict' object has no attribute 'set_index'

In [61]:
conv

{'Egypt, Arab Rep.': 'Egypt',
 'Hong Kong SAR, China': 'Hong Kong',
 'Iran, Islamic Rep.': 'Iran',
 'Macao SAR, China': 'Macau',
 'China': 'Mainland China',
 'West Bank and Gaza': 'Palestine',
 'Russian Federation': 'Russia',
 'Korea, Rep.': 'South Korea',
 'United Kingdom': 'UK',
 'United States': 'US'}

In [62]:
df.replace({"country": conv})

,country,country_code,population,region,income
0,Aruba,ABW,1.058450e+05,Latin America & Caribbean,High income
1,Afghanistan,AFG,3.717239e+07,South Asia,Low income
2,Angola,AGO,3.080976e+07,Sub-Saharan Africa,Lower middle income
3,Albania,ALB,2.866376e+06,Europe & Central Asia,Upper middle income
4,Andorra,AND,7.700600e+04,Europe & Central Asia,High income
5,Arab World,ARB,4.197906e+08,NaN,NaN
6,United Arab Emirates,ARE,9.630959e+06,Middle East & North Africa,High income
7,Argentina,ARG,4.449450e+07,Latin America & Caribbean,Upper middle income
8,Armenia,ARM,2.951776e+06,Europe & Central Asia,Upper middle income
9,American Samoa,ASM,5.546500e+04,East Asia & Pacific,Upper middle income


In [66]:
df = pd.read_csv('../combined_ts.csv')
df.head()

,country,state,date,confirmed,deaths,recovered
0,Mainland China,Anhui,2020-01-22,1,0,0
1,Mainland China,Beijing,2020-01-22,14,0,0
2,Mainland China,Chongqing,2020-01-22,6,0,0
3,Mainland China,Fujian,2020-01-22,1,0,0
4,Mainland China,Gansu,2020-01-22,0,0,0


In [73]:
pdf = pd.read_csv('../pop.csv').set_index('country')
pdf.head()

,country_code,population,region,income
country,,,,
Aruba,ABW,105845.0,Latin America & Caribbean,High income
Afghanistan,AFG,37172386.0,South Asia,Low income
Angola,AGO,30809762.0,Sub-Saharan Africa,Lower middle income
Albania,ALB,2866376.0,Europe & Central Asia,Upper middle income
Andorra,AND,77006.0,Europe & Central Asia,High income


In [75]:
meta_dict = pdf.to_dict()

In [77]:
data = []
for group in df.groupby(["country"]):
    # ignore regions within a country. TODO: incorporate regions.
    if group[0] == 'Others':
        continue
    temp_df = (
        group[1]
        .groupby(["date"])[["confirmed", "deaths", "recovered"]]
        .sum()
        .reset_index()
    )
    country_data = {
        "country": group[0],
        "population": meta_dict['population'][group[0]],
        "region": meta_dict['region'][group[0]],
        "income": meta_dict['income'][group[0]],
        "confirmed": temp_df[["date", "confirmed"]].values.tolist(),
        "deaths": temp_df[["date", "deaths"]].values.tolist(),
        "recovered": temp_df[["date", "recovered"]].values.tolist(),
    }
    data.append(country_data)
data

[{'country': 'Afghanistan',
  'population': 37172386.0,
  'region': 'South Asia',
  'income': 'Low income',
  'confirmed': [['2020-01-22', 0],
   ['2020-01-23', 0],
   ['2020-01-24', 0],
   ['2020-01-25', 0],
   ['2020-01-26', 0],
   ['2020-01-27', 0],
   ['2020-01-28', 0],
   ['2020-01-29', 0],
   ['2020-01-30', 0],
   ['2020-01-31', 0],
   ['2020-02-01', 0],
   ['2020-02-02', 0],
   ['2020-02-03', 0],
   ['2020-02-04', 0],
   ['2020-02-05', 0],
   ['2020-02-06', 0],
   ['2020-02-07', 0],
   ['2020-02-08', 0],
   ['2020-02-09', 0],
   ['2020-02-10', 0],
   ['2020-02-11', 0],
   ['2020-02-12', 0],
   ['2020-02-13', 0],
   ['2020-02-14', 0],
   ['2020-02-15', 0],
   ['2020-02-16', 0],
   ['2020-02-17', 0],
   ['2020-02-18', 0],
   ['2020-02-19', 0],
   ['2020-02-20', 0],
   ['2020-02-21', 0],
   ['2020-02-22', 0],
   ['2020-02-23', 0],
   ['2020-02-24', 1],
   ['2020-02-25', 1],
   ['2020-02-26', 1],
   ['2020-02-27', 1],
   ['2020-02-28', 1],
   ['2020-02-29', 1],
   ['2020-03-01', 1],